0. Chuẩn bị: dataset toy + tiện ích


In [ ]:
!pip install torch torchvision scikit-learn


DEPRECATION: Loading egg at c:\users\admin\miniconda3\envs\myenvironment\lib\site-packages\bs4-0.0.2-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\admin\miniconda3\envs\myenvironment\lib\site-packages\lxml-5.3.0-py3.13-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\admin\miniconda3\envs\myenvironment\lib\site-packages\narwhals-1.19.0-py3.13.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\admin\miniconda3\envs\myenvironment\l


  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 1.6/241.3 MB 12.8 MB/s eta 0:00:19
   -- ------------------------------------- 14.7/241.3 MB 46.9 MB/s eta 0:00:05
   --- ------------------------------------ 23.3/241.3 MB 54.2 MB/s eta 0:00:05
   ----- ---------------------------------- 34.3/241.3 MB 48.8 MB/s eta 0:00:05
   ------ --------------------------------- 41.2/241.3 MB 45.0 MB/s eta 0:00:05
   ------- -------------------------------- 46.7/241.3 MB 42.3 MB/s eta 0:00:05
   -------- ------------------------------- 50.6/241.3 MB 38.3 MB/s eta 0:00:05
   -------- ------------------------------- 53.5/241.3 MB 35.4 MB/s eta 0:00:06
   --------- ------------------------------ 56.6/241.3 MB 33.0 MB/s eta 0:00:06
   --------- ------------------------------ 59.8/241.3 MB 31.2 MB/s eta 0:00:06
   ---------- ----------------------------- 63.2/241.3 MB 29.7 MB/

In [ ]:
# !pip install torch torchvision scikit-learn
import math
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Toy classification data (tabular) để demo nhanh


def make_toy(n=5000, d=20, n_classes=4, seed=0):
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(n, d))
    W = rng.normal(size=(d, n_classes))
    logits = X @ W + rng.normal(scale=0.5, size=(n, n_classes))
    y = logits.argmax(axis=1)
    return X.astype(np.float32), y.astype(np.int64)


X, y = make_toy()


1. Train/Dev/Test split (tránh leakage)


In [ ]:
# Stratified split để giữ tỉ lệ lớp, dev nhỏ (10%) nếu data lớn
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)
X_dev,   X_test, y_dev,   y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Nếu có "group" (ví dụ: cùng user), dùng GroupShuffleSplit để không rò rỉ theo người dùng.
# from sklearn.model_selection import GroupShuffleSplit
# gss = GroupShuffleSplit(test_size=0.3, random_state=42)
# train_idx, temp_idx = next(gss.split(X, y, groups=user_ids))


2. Chuẩn hoá đầu vào (không leak: fit trên train, transform trên dev/test)


In [ ]:
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_dev_s = scaler.transform(X_dev)
X_test_s = scaler.transform(X_test)


3. MLP nhỏ với L2 (weight_decay), Dropout, BatchNorm, He init


In [ ]:
class MLP(nn.Module):
    def __init__(self, d_in, d_hidden=256, d_out=4, p_drop=0.5, use_bn=True, act='relu'):
        super().__init__()
        self.fc1 = nn.Linear(d_in, d_hidden)
        self.bn1 = nn.BatchNorm1d(d_hidden) if use_bn else nn.Identity()
        self.drop = nn.Dropout(p_drop)
        self.fc2 = nn.Linear(d_hidden, d_out)
        self.act = nn.ReLU() if act == 'relu' else nn.Tanh()
        self._init_weights(act)

    def _init_weights(self, act):
        # He cho ReLU, Xavier cho tanh
        if act == 'relu':
            nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        else:
            nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)     # logits cho Softmax đa lớp
        return x

# Tensor + DataLoader (mini-batch)


def make_loader(X, y, batch_size=128, shuffle=True):
    ds = TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)


4. Vòng lặp train tổng quát + optimizers (SGD+momentum/RMSprop/Adam) + clip grad


In [ ]:
def train_model(X_train, y_train, X_dev, y_dev, *,
                lr=1e-3, weight_decay=1e-4, batch_size=128,
                optimizer_name='adam', max_epochs=20, clip_norm=5.0):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = MLP(X_train.shape[1], d_hidden=256, d_out=len(
        np.unique(y_train)), p_drop=0.5, use_bn=True).to(device)

    if optimizer_name == 'sgd':
        opt = torch.optim.SGD(model.parameters(), lr=lr,
                              momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == 'rmsprop':
        opt = torch.optim.RMSprop(
            model.parameters(), lr=lr, alpha=0.99, weight_decay=weight_decay)
    else:  # 'adam'
        opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(
            0.9, 0.999), eps=1e-8, weight_decay=weight_decay)

    # Lịch giảm LR (step decay ví dụ)
    scheduler = torch.optim.lr_scheduler.StepLR(
        opt, step_size=max(1, max_epochs//3), gamma=0.1)

    loss_fn = nn.CrossEntropyLoss()
    train_loader = make_loader(
        X_train, y_train, batch_size=batch_size, shuffle=True)
    dev_loader = make_loader(X_dev,   y_dev,   batch_size=1024, shuffle=False)

    for epoch in range(1, max_epochs+1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss = loss_fn(logits, yb)
            opt.zero_grad()
            loss.backward()
            # Chống exploding gradients
            if clip_norm is not None:
                nn.utils.clip_grad_norm_(
                    model.parameters(), max_norm=clip_norm)
            opt.step()
        scheduler.step()

    # Evaluate dev
    model.eval()
    with torch.no_grad():
        xb = torch.from_numpy(X_dev).to(device)
        y_pred = model(xb).argmax(dim=1).cpu().numpy()
    dev_acc = accuracy_score(y_dev, y_pred)
    return model, dev_acc

# Ví dụ chạy nhanh (bỏ nếu không cần chạy):
# model, dev_acc = train_model(X_train_s, y_train, X_dev_s, y_dev, optimizer_name='adam')
# print("Dev acc:", dev_acc)


5. L1 regularization (thêm tay vào loss)


In [ ]:
def l1_penalty(model):
    return sum(p.abs().sum() for p in model.parameters())

# Trong vòng lặp train, thay:
# loss = loss_fn(logits, yb) + lam_l1 * l1_penalty(model)


6. Gradient Checking (NumPy, central differences) – dùng để debug backprop


In [ ]:
# Ví dụ network 1 hidden (tanh), softmax CE – chỉ để GRAD-CHECK (không tối ưu hiệu năng)
def softmax(z):
    z = z - z.max(axis=1, keepdims=True)
    ez = np.exp(z)
    return ez / ez.sum(axis=1, keepdims=True)


def forward_backward(params, X, y):
    # params: dict chứa W1,b1,W2,b2
    W1, b1, W2, b2 = params['W1'], params['b1'], params['W2'], params['b2']
    n = X.shape[0]
    # forward
    Z1 = X @ W1 + b1
    A1 = np.tanh(Z1)
    Z2 = A1 @ W2 + b2
    P = softmax(Z2)
    # loss CE
    Y = np.eye(W2.shape[1])[y]
    loss = -np.sum(Y * np.log(P + 1e-12)) / n
    # backward
    dZ2 = (P - Y) / n
    dW2 = A1.T @ dZ2
    db2 = dZ2.sum(axis=0, keepdims=True)
    dA1 = dZ2 @ W2.T
    dZ1 = dA1 * (1 - np.tanh(Z1)**2)
    dW1 = X.T @ dZ1
    db1 = dZ1.sum(axis=0, keepdims=True)
    grads = {'W1': dW1, 'b1': db1, 'W2': dW2, 'b2': db2}
    return loss, grads


def numerical_grad(params, X, y, eps=1e-7):
    num = {}
    for k in params:
        num[k] = np.zeros_like(params[k])
        it = np.nditer(params[k], flags=['multi_index'],
                       op_flags=['readwrite'])
        while not it.finished:
            idx = it.multi_index
            old = params[k][idx]
            params[k][idx] = old + eps
            lp, _ = forward_backward(params, X, y)
            params[k][idx] = old - eps
            lm, _ = forward_backward(params, X, y)
            num[k][idx] = (lp - lm) / (2*eps)
            params[k][idx] = old
            it.iternext()
    return num


def relative_diff(ga, gn):
    num = sum(np.linalg.norm(ga[k] - gn[k]) for k in ga)
    den = sum(np.linalg.norm(ga[k]) + np.linalg.norm(gn[k]) for k in ga)
    return num / (den + 1e-12)


# Demo nhỏ:
np.random.seed(0)
d_in, d_h, d_out = 5, 4, 3
X_small = np.random.randn(6, d_in)
y_small = np.random.randint(0, d_out, size=6)
params = {
    'W1': np.random.randn(d_in, d_h)*0.1, 'b1': np.zeros((1, d_h)),
    'W2': np.random.randn(d_h, d_out)*0.1, 'b2': np.zeros((1, d_out)),
}
loss, grads = forward_backward(params, X_small, y_small)
numg = numerical_grad({k: v.copy()
                      for k, v in params.items()}, X_small, y_small)
# ~ 1e-6 hoặc nhỏ hơn là ổn
print("relative diff:", relative_diff(grads, numg))


relative diff: 1.698930842107017e-08


7. Học theo mini-batch + chọn optimizer

Đã thể hiện trong train_model(...) ở mục (4). Chỉ cần đổi optimizer_name giữa 'sgd' | 'rmsprop' | 'adam'.

Nếu bạn muốn RMSProp + Momentum kiểu Nadam/AdamW, thay optimizer tương ứng:


In [ ]:
# AdamW (decoupled weight decay – thường tốt hơn với transformer, CNN lớn)
opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)


NameError: name 'model' is not defined

8. Lịch Learning Rate (cosine, one-cycle, v.v.)


In [ ]:
# Cosine annealing
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=10)

# OneCycleLR (tăng LR rồi giảm – thường hợp khi train từ scratch trên image)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=lr, steps_per_epoch=len(train_loader), epochs=max_epochs)


NameError: name 'opt' is not defined

9. Random Search HPO (log-scale cho LR, coarse→fine)


In [ ]:
def sample_hparams():
    # log-uniform cho LR: 10^U[-4, -2] = [1e-4, 1e-2]
    lr = 10 ** np.random.uniform(-4, -2)
    weight_decay = 10 ** np.random.uniform(-6, -3)    # L2
    batch_size = int(2 ** np.random.uniform(5, 8))     # 32..256
    p_drop = np.random.uniform(0.1, 0.6)
    opt_name = random.choice(['adam', 'rmsprop', 'sgd'])
    return dict(lr=lr, weight_decay=weight_decay, batch_size=batch_size, p_drop=p_drop, optimizer_name=opt_name)


def run_trial(Xtr, ytr, Xdv, ydv, trial_id=0, max_epochs=15):
    hp = sample_hparams()
    # cập nhật dropout cho model bằng cách monkeypatch tham số khi khởi tạo
    model, dev_acc = train_model(Xtr, ytr, Xdv, ydv,
                                 lr=hp['lr'], weight_decay=hp['weight_decay'],
                                 batch_size=hp['batch_size'],
                                 optimizer_name=hp['optimizer_name'],
                                 max_epochs=max_epochs)
    return dev_acc, hp


best = (-1, None)
for t in range(10):  # coarse search 10 trial, sau đó zoom-in vùng tốt
    acc, hp = run_trial(X_train_s, y_train, X_dev_s,
                        y_dev, trial_id=t, max_epochs=12)
    if acc > best[0]:
        best = (acc, hp)
print("Best dev acc:", best)


Best dev acc: (0.896, {'lr': 0.0014187079131473233, 'weight_decay': 0.0006136045802294309, 'batch_size': 62, 'p_drop': 0.4337051899818408, 'optimizer_name': 'rmsprop'})


10. Đọc xác suất Softmax + dự đoán


In [ ]:
def predict_proba(model, X):
    model.eval()
    with torch.no_grad():
        logits = model(torch.from_numpy(X))
        probs = torch.softmax(logits, dim=1).numpy()
    return probs

# probs_dev = predict_proba(model, X_dev_s)
# y_pred = probs_dev.argmax(axis=1)
# print("Dev accuracy:", accuracy_score(y_dev, y_pred))


11. “Nếu thấy triệu chứng thì làm gì?” (áp dụng vào code)

Train loss cao & Dev loss cao (High Bias)
→ tăng d_hidden, thêm layer, đổi activation, train lâu hơn, giảm weight_decay, bỏ bớt Dropout.

Train loss thấp & Dev loss cao (High Variance)
→ tăng weight_decay, tăng p_drop, thêm/đổi augmentation (cho ảnh), thu nhỏ model, thu thêm dữ liệu.

Loss “răng cưa”, không ổn định
→ giảm lr hoặc dùng Adam/RMSprop; tăng batch_size; dùng BatchNorm; clip grad.

Học chậm
→ chuẩn hoá đúng (fit trên train), He init (ReLU) / Xavier (tanh), dùng scheduler LR
